In [1]:
import sys
import pandas as pd
from gurobipy import *
import time

In [2]:
start_code = time.time()

In [3]:
data = pd.read_excel('发货.xls')
dist = pd.read_csv('inputdistancetime_5_1201.csv')
node = list(data['ID'])
data.set_index(["ID"], inplace=True)

In [4]:
s = len(data[data['type'] == 4])  #充电桩个数

In [5]:
s

100

In [6]:
#电动车状态
Battery = 120000.0
Capacity = 2.5
Volume = 16
Consume = 1.0
SwapTime = 30.0

Demand = {}
Vol_Demand = {}
ReaTime = {}
DueTime = {}
SerTime = 30.0
Distance = {}
Time = {}

In [7]:
for i in node:
    Demand[i] = data.loc[i,'pack_total_weight']
    Vol_Demand[i] = data.loc[i,'pack_total_volume']
    ReaTime[i] = int(data.loc[i,'first_receive_tm'])
    DueTime[i] = int(data.loc[i,'last_receive_tm'])

In [14]:
ReaTime[50901]

450

In [15]:
#距离和时间
node.append(2000)
for i in dist.index:
    Distance[dist.loc[i,'from_node'],dist.loc[i,'to_node']] = dist.loc[i,'distance']
    Time[dist.loc[i,'from_node'],dist.loc[i,'to_node']] = dist.loc[i,'spend_tm']

In [16]:
Time[0,50001]

68

In [19]:
node[100]

51000

In [20]:
##回来的点
for i in node:
    if i not in [0,2000]:
        Distance[2000,i] = Distance[0,i]
        Distance[i,2000] = Distance[i,0]
        Time[i,2000] = Time[i,0]
        Time[2000,i] = Time[0,i]
Distance[0,2000] = Distance[2000,0] = 0
Time[0,2000] = Time[2000,0] = 0

In [21]:
Distance[50001,0]

58509

In [22]:
Demand[2000] = Demand[0]
Vol_Demand[2000] = Vol_Demand[0]
ReaTime[2000] = ReaTime[0]
DueTime[2000] = DueTime[0]
n = len(node)
print(node)
print('data ok')

[0, 50901, 50902, 50903, 50904, 50905, 50906, 50907, 50908, 50909, 50910, 50911, 50912, 50913, 50914, 50915, 50916, 50917, 50918, 50919, 50920, 50921, 50922, 50923, 50924, 50925, 50926, 50927, 50928, 50929, 50930, 50931, 50932, 50933, 50934, 50935, 50936, 50937, 50938, 50939, 50940, 50941, 50942, 50943, 50944, 50945, 50946, 50947, 50948, 50949, 50950, 50951, 50952, 50953, 50954, 50955, 50956, 50957, 50958, 50959, 50960, 50961, 50962, 50963, 50964, 50965, 50966, 50967, 50968, 50969, 50970, 50971, 50972, 50973, 50974, 50975, 50976, 50977, 50978, 50979, 50980, 50981, 50982, 50983, 50984, 50985, 50986, 50987, 50988, 50989, 50990, 50991, 50992, 50993, 50994, 50995, 50996, 50997, 50998, 50999, 51000, 51001, 51002, 51003, 51004, 51005, 51006, 51007, 51008, 51009, 51010, 51011, 51012, 51013, 51014, 51015, 51016, 51017, 51018, 51019, 51020, 51021, 51022, 51023, 51024, 51025, 51026, 51027, 51028, 51029, 51030, 51031, 51032, 51033, 51034, 51035, 51036, 51037, 51038, 51039, 51040, 51041, 51042, 51

In [23]:
n

302

In [24]:










m = Model('VRP')
m.setParam('OutputFlag',True)########
m.setParam('TimeLimit',43200.0)

Academic license - for non-commercial use only
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TimeLimit to 43200.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100


In [25]:
###构造变量
#travle or not
x = {}           
for i in node:
    for j in node:
        if i != j:
            x[i,j] = m.addVar(0.0, 1.0, 0.0, GRB.BINARY,'x_%s%s' % (i, j))

In [26]:
#到达时间
ArrTime = {}
for i in node:
    ArrTime[i] = m.addVar(ReaTime[i], DueTime[i], 0.0, GRB.CONTINUOUS, "ArrTime_%s" %i)

In [27]:
#剩余货物
RemainCargo = {}
RemainVelocity = {}
for i in node:
    RemainCargo[i] = m.addVar(0,Capacity,0.0,GRB.CONTINUOUS,'RemainCargo_%s' %i)
for i in node:
    RemainVelocity[i] = m.addVar(0,Volume,0.0,GRB.CONTINUOUS,'RemainVelocity_%s' %i)

In [28]:
#剩余电量
RemainBattery = {}
for i in node:
    RemainBattery[i] = m.addVar(0,Battery, 0.0, GRB.CONTINUOUS,'RemainBattery_%s' %i)
#Integrate new variables
    m.update()

In [29]:
#构造目标函数
obj1 = quicksum(x[0,j] for j in node[1:n]) #车辆数
obj2 = quicksum(Distance[i,j]*x[i,j] for i in node for j in node[1:n] if i != j)  #运输距离
obj3 = quicksum(x[i,j] for i in node[1:s+1] for j in node[s+1:n])      #充电次数

In [30]:
m.setObjective(50*obj3 + 0.014*obj2 + 300*obj1,GRB.MINIMIZE)

In [31]:
##构造约束条件
#1.每个顾客点都被访问
for i in node[s+1:n-1]:
    m.addConstr(quicksum(x[i,j] for j in node[1:n] if i != j) == 1) 

In [32]:
#2.虚拟充电站访问设置
for i in node[1:s+1]:
    m.addConstr(quicksum(x[i,j] for j in node[1:n] if i != j) <= 5) 

In [33]:
#3.从A点进入了就要从A点出去
for h in node[1:n-1]:
    m.addConstr(quicksum(x[h,i] for i in node[1:n] if i != h) - quicksum(x[j,h] for j in node[0:n-1] if h != j) == 0)

In [34]:
#4.时间窗
for j in node[1:n]:
    m.addConstr(ArrTime[0]+(Time[0,j]+0)*x[0,j]-DueTime[0]*(1-x[0,j])-ArrTime[j]<=0)
for i in node[s+1:n-1]:
    for j in node[1:n]:
        if i != j:
            m.addConstr(ArrTime[i]+(Time[i,j]+SerTime)*x[i,j]-DueTime[0]*(1-x[i,j])-ArrTime[j]<=0)
for i in node[1:s+1]:
    for j in node[1:n]:
        if i != j:
            m.addConstr(ArrTime[i]+(Time[i,j]+SwapTime)*x[i,j]-(DueTime[0]+SwapTime)*(1-x[i,j])-ArrTime[j]<=0)

In [35]:
#5.容量限制
for i in node[0:n-1]:
    for j in node[1:n]:
        if i != j:
            m.addConstr(RemainCargo[j]-RemainCargo[i]+Demand[i]*x[i,j]-Capacity*(1-x[i,j])<=0)
for i in node[0:n-1]:
    for j in node[1:n]:
        if i != j:
            m.addConstr(RemainVelocity[j]-RemainVelocity[i]+Vol_Demand[i]*x[i,j]-Volume*(1-x[i,j])<=0)

In [36]:
#6.电池容量限制
for i in node[s+1:n-1]:
    for j in node[1:n]:
        if i != j:
            m.addConstr(RemainBattery[j]-RemainBattery[i]+Consume*Distance[i,j]*x[i,j]-Battery*(1-x[i,j])<=0)
for j in node[1:n]:
    m.addConstr(RemainBattery[j]-RemainBattery[0]+Consume*Distance[0,j]*x[0,j]-Battery*(1-x[0,j])<=0)

In [37]:
for i in node[1:s+1]:
    for j in node[1:len(node)]:
        if i != j:
            m.addConstr(RemainBattery[j] +Consume*Distance[i,j]*x[i,j]-Battery<=0)     

In [38]:
#7.其他的一些限制
for i in node[1:n-1]:
    m.addConstr(x[i,0] == 0)
    m.addConstr(x[2000,i] == 0)

In [ ]:
#m.addConstr(x[0,node[1]] == 0)
#m.addConstr(x[node[1],2000] == 0)

In [39]:
m.update()

In [40]:
try:
    m.optimize()
    print(m.ObjVal)
except GurobiError:
    print('Error reported')
print("optimize()")
print(GRB.Status)

Optimize a model with 362404 rows, 92110 columns and 1324212 nonzeros
Variable types: 1208 continuous, 90902 integer (90902 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [3e-01, 1e+03]
  Bounds range     [1e+00, 1e+05]
  RHS range        [1e+00, 1e+05]
Presolve removed 103963 rows and 24956 columns
Presolve time: 2.55s
Presolved: 258441 rows, 67154 columns, 876275 nonzeros
Variable types: 1100 continuous, 66054 integer (65954 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     953    1.0301822e+04   1.653504e+05   0.000000e+00      5s
    1637    1.0443132e+04   0.000000e+00   0.000000e+00      6s

Root relaxation: objective 1.044313e+04, 1637 iterations, 1.26 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 10443.1325    0  344          - 10443.1325      -     -    8s
     0    

   268   269 12594.8499   65  492          - 12360.9356      -   110  726s
   288   290 12644.6375   72  475          - 12360.9356      -   109  740s
   308   306 12655.1238   78  480          - 12360.9356      -   108  755s
   326   328 12672.2354   84  483          - 12360.9356      -   108  769s
   344   346 12758.8499   88  456          - 12360.9356      -   110  789s
   369   369 12767.9306   91  451          - 12360.9356      -   109  812s
   391   394 12819.3986   95  450          - 12360.9356      -   108  832s
   414   415 12839.9043   98  437          - 12360.9356      -   109  850s
   434   433 12880.1237  105  438          - 12360.9356      -   109  869s
   465   468 12948.6916  108  431          - 12360.9356      -   110  889s
   491   492 12985.9158  111  438          - 12360.9356      -   111  906s
   528   528 13138.0823  114  444          - 12360.9356      -   111  926s
   566   565 13159.2678  121  421          - 12360.9356      -   110  949s
   610   616 13234.1313  

  5296  5271 15639.3238  334  612          - 13105.7677      -  95.4 5545s
  5297  5272 15027.0216  312  623          - 13116.6490      -  95.4 5586s
  5298  5273 14717.3720  233  643          - 13120.0816      -  95.4 5595s
  5299  5273 14272.1305  234  633          - 13121.4647      -  95.4 5607s
  5300  5274 19008.2861  725  653          - 13121.8934      -  95.4 5611s
  5301  5275 18691.0982  886  654          - 13121.9663      -  95.3 5668s
  5302  5275 13144.6036   34  635          - 13144.6036      -  95.3 5769s
  5303  5276 18473.3740  907  637          - 13149.4809      -  95.3 5779s
  5304  5277 18407.7158  600  638          - 13150.8818      -  95.3 5786s
  5305  5277 18482.5108  798  650          - 13151.7009      -  95.3 5791s
  5306  5278 19877.7146 1014  655          - 13151.7792      -  95.2 5851s
  5307  5279 17118.1177  708  613          - 13172.1078      -  95.2 5956s
  5308  5279 20543.2386  934  656          - 13179.0457      -  95.2 5971s
  5309  5280 19654.5624  

  5407  4582 17118.1177  708  728 73966.8280 13438.7289  81.8%  93.5 11758s
  5408  4583 20543.2386  934  725 73966.8280 13438.8153  81.8%  93.5 11762s
  5409  4584 19654.5624  996  726 73966.8280 13438.8287  81.8%  93.4 11810s
  5410  4584 18545.9276  840  710 73966.8280 13444.0562  81.8%  93.4 11875s
  5411  4585 13979.7566  162  712 73966.8280 13444.8040  81.8%  93.4 11880s
  5413  4586 20183.1180 1025  722 73966.8280 13445.0456  81.8%  93.4 11887s
  5414  4587 15500.1113  392  721 73966.8280 13445.1522  81.8%  93.3 11891s
  5415  4588 17421.3095  747  717 73966.8280 13445.2299  81.8%  93.3 11940s
  5416  4588 13452.8787   35  673 73966.8280 13452.8787  81.8%  93.3 12034s
  5417  4589 14168.5206  200  709 73966.8280 13454.3838  81.8%  93.3 12046s
  5418  4590 19922.2257 1029  696 73966.8280 13454.9196  81.8%  93.3 12052s
  5419  4590 20957.8093 1004  698 73966.8280 13455.2060  81.8%  93.3 12057s
  5420  4591 18621.9623  633  707 73966.8280 13455.6152  81.8%  93.2 12062s
  5421  4592

  5517  4656 14168.5206  200  746 73966.8280 13524.5206  81.7%  91.6 28596s
  5518  4656 19922.2257 1029  731 73966.8280 13525.6293  81.7%  91.6 28664s
  5519  4657 20957.8093 1004  738 73966.8280 13525.7215  81.7%  91.6 28669s
  5520  4658 18621.9623  633  748 73966.8280 13525.7467  81.7%  91.6 28726s
  5521  4658 20585.0292 1144  724 73966.8280 13526.3269  81.7%  91.5 28798s
  5522  4659 13578.3487  174  731 73966.8280 13526.8733  81.7%  91.5 28805s
  5523  4660 14641.4371  214  750 73966.8280 13527.0956  81.7%  91.5 28811s
  5524  4660 18474.5860  793  759 73966.8280 13527.1438  81.7%  91.5 28869s
  5525  4661 18204.0764  839  742 73966.8280 13527.6809  81.7%  91.5 28965s
  5526  4662 13527.9262  143  741 73966.8280 13527.9262  81.7%  91.5 28971s
  5527  4662 18597.2497  933  747 73966.8280 13527.9748  81.7%  91.4 29031s
  5528  4663 19502.6858  929  726 73966.8280 13528.4797  81.7%  91.4 29189s
  5529  4664 18372.2352  734  730 73966.8280 13528.6678  81.7%  91.4 29194s
  5530  4664

  5627  4268 18597.2497  933  723 73887.6240 13556.6064  81.7%   115 38558s
  5628  4269 19502.6858  929  723 73887.6240 13556.6064  81.7%   114 39631s
  5629  4272 13563.1265   28  713 73887.6240 13563.1265  81.6%   121 41054s
  5631  4276 13567.0349   29  703 73887.6240 13565.6580  81.6%   121 41536s
H 5633  4061                    73801.258000 13565.6580  81.6%   121 41537s
  5635  4064 13573.2804   30  709 73801.2580 13566.4271  81.6%   122 41609s
  5639  4067 13584.2090   30  688 73801.2580 13567.2871  81.6%   122 41681s
  5643  4070 13575.5304   31  678 73801.2580 13567.2871  81.6%   123 41729s
  5647  4072 13575.3535   31  695 73801.2580 13567.3099  81.6%   123 41782s
  5651  4075 13580.6364   32  652 73801.2580 13567.3099  81.6%   123 41829s

Cutting planes:
  Learned: 9
  Gomory: 19
  Cover: 26
  Implied bound: 34
  Clique: 7
  MIR: 131
  StrongCG: 4
  Flow cover: 637
  GUB cover: 5
  Zero half: 150

Explored 5655 nodes (713300 simplex iterations) in 43201.17 seconds
Thread co

In [41]:
def printTour(solution, route, start, distance, demand,vol_demand,sol_time,solved_point,point):
    route = route + str(start) + ' -> '
    if start not in node[0:s+1]:
        solved_point.append(start)
    point.append(start)
    demand += Demand[start]
    vol_demand +=Vol_Demand[start]
    for i in node:
        if start != i:
            if (solution[start, i]> 0.5) :
                if i not in solved_point and (round(sol_time[i])>=round(sol_time[start]+30+Time[start,i])): 
                    totalDistance = distance + Distance[start,i]
                    if (i == 2000):
                        return [
                        totalDistance,
                        demand,
                        vol_demand,
                        route + '2000',
                        start,        #最后一个点
                        point,
                        solved_point
                        ]
                    return printTour(solution, route, i, totalDistance, demand,vol_demand,sol_time,solved_point,point)         

In [43]:
E = pd.DataFrame()
if True:
    print('objective: %f' % m.ObjVal)
    solution = m.getAttr('x', x)
    sol_time = m.getAttr("x", ArrTime)

    solved_point = []
    for i in node:
        if i != 0:
            if solution[0,i] >0.5:
                start_time = sol_time[i]-Time[0,i]
                output = printTour(solution,'0 ->',i,Distance[0,i],0,0,sol_time,solved_point,[0])
                solved_point = output[6]                
                points = output[5]
                charge_time = 0
                for p in points:
                    if p in node[1:s+1]:
                        charge_time = charge_time+1
                output = output[0:6] +[start_time,charge_time]  
                if output[0]<=100000 and output[1]<=2 and output[2]<=12:
                    output.append(1)
                else:
                    output.append(2)
                E = E.append([output])

    E.columns= [u'里程',u'总重量',u'总体积',u'顺序',u'最后一点',u'经过的点',u'出发时间',u'充电次数',u'车型']  
    E.to_csv('out.csv',encoding = 'utf-8')

objective: 73801.258000


In [44]:
end_code = time.time()
print(start_code-end_code)

-81896.14847755432
